# Versión 2 algoritmos posiciones y comparación
## Autor: José Miguel Ramírez Sanz
## Descripción: notebook donde se encuentra la versión final no reducida de la posicion y las pruebas temporales

In [1]:
# imports
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import math
import os
from datetime import datetime as dt
import pandas as pd
import pickle as pk
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

# import de utilities de detectron
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

### Carga del modelo

In [2]:
#Carga del modelo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.99  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

### Versión 2 de la posicion, es la versión final sin reducir

In [3]:
#Posiciones versión 2
class Posicion():
    
    def __init__(self,x,y):
        self.nariz = [x[0],y[0]]
        self.ojoI = [x[1],y[1]]
        self.ojoD = [x[2],y[2]]
        self.orejaI=[x[3],y[3]]
        self.orejaD=[x[4],y[4]]
        self.hombroI=[x[5],y[5]]
        self.hombroD=[x[6],y[6]]
        self.cuello = self.calcularPuntoMedio(self.hombroI,self.hombroD)
        self.angCuelloSupI = self.calcularAuxAngulo(self.hombroI,self.cuello,self.nariz)
        self.angCuelloSupD = self.calcularAuxAngulo(self.hombroD,self.cuello,self.nariz)
        self.codoI = [x[7],y[7]]
        self.codoD = [x[8],y[8]]
        self.manoI=[x[9],y[9]]
        self.manoD = [x[10],y[10]]
        self.angCodoI = self.calcularAngulo(self.hombroI,self.codoI,self.manoI,0)
        self.angCodoD = self.calcularAngulo(self.hombroD,self.codoD,self.manoD,1)
        self.angHombroI = self.calcularAngulo(self.cuello,self.hombroI,self.codoI,0)
        self.angHombroD = self.calcularAngulo(self.cuello,self.hombroD,self.codoD,1)
        self.caderaI = [x[11],y[11]]
        self.caderaD = [x[12],y[12]]
        self.cadera = self.calcularPuntoMedio(self.caderaI,self.caderaD)
        self.angCuelloInfI =self.calcularAuxAngulo(self.hombroI,self.cuello,self.cadera)
        self.angCuelloInfD =self.calcularAuxAngulo(self.hombroD,self.cuello,self.cadera)
        self.rodillaI = [x[13],y[13]]
        self.rodillaD = [x[14],y[14]]
        self.angCaderaI = self.calcularAngulo(self.cadera,self.caderaI,self.rodillaI,0)
        self.angCaderaD = self.calcularAngulo(self.cadera,self.caderaD,self.rodillaD,1)
        self.angCaderaTorsoI = self.calcularAuxAngulo(self.cuello,self.cadera,self.caderaI)
        self.angCaderaTorsoD = self.calcularAuxAngulo(self.cuello,self.cadera,self.caderaD)
        self.tobilloI = [x[15],y[15]]
        self.tobilloD = [x[16],y[16]]
        self.angRodillaI = self.calcularAngulo(self.caderaI,self.rodillaI,self.tobilloI,0)
        self.angRodillaD = self.calcularAngulo(self.caderaD,self.rodillaD,self.tobilloD,1)
        self.distAntebrazoI = self.calcularDistancia(self.manoI,self.codoI)
        self.distAntebrazoD = self.calcularDistancia(self.manoD,self.codoD)
        self.distBrazoI = self.calcularDistancia(self.codoI,self.hombroI)
        self.distBrazoD = self.calcularDistancia(self.codoD,self.hombroD)
        self.distCuello = self.calcularDistancia(self.nariz,self.cuello)
        self.distTronco = self.calcularDistancia(self.cuello,self.cadera)
        self.distMusloI = self.calcularDistancia(self.caderaI,self.rodillaI)
        self.distMusloD = self.calcularDistancia(self.caderaD,self.rodillaD)
        self.distPiernaI = self.calcularDistancia(self.rodillaI,self.tobilloI)
        self.distPiernaD = self.calcularDistancia(self.rodillaD,self.tobilloD)
        self.distHombros = self.calcularDistancia(self.hombroI,self.hombroD)
        self.distCadera = self.calcularDistancia(self.caderaI,self.caderaD)
        
    def calcularPuntoMedio(self,p1,p2):
        return [(p1[0]+p2[0])/2,(p1[1]+p2[1])/2]
    
    def calcularAuxAngulo(self,p1,p2,p3):
        v1 = self.calcularVector(p1,p2)
        v2 = self.calcularVector(p3,p2)
        uv1 = v1 / np.linalg.norm(v1)
        uv2 = v2 / np.linalg.norm(v2)
        dp = np.dot(uv1, uv2)
        if dp > 1:
            dp=1
        elif dp < -1:
            dp=-1
        return math.degrees(np.arccos(dp))
    
    def calcularVector(self,p1,p2):
        return [p2[0]-p1[0],p2[1]-p1[1]]
    
    def calcularDistancia(self,p1,p2):
        return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5
    
    def calcularAngulo(self,p1,p2,p3,lado):
        flag=False
        ang = self.calcularAuxAngulo(p1,p2,p3)
        
        #Cuando la recta que une los dos primeros puntos es una línea vertical
        if (p2[0]-p1[0]) ==0:
            #lado derecho
            if lado:
                #Si la posicion está menos a la derecha se ha de cambiar el ángulo
                if p3[0]<p2[0]:
                    flag=True
            #lado izquierdo
            else:
                #Si la posicion está menos a la izquierda se ha de cambiar el ángulo
                if p3[0]>p2[0]:
                    flag=True
            #Cambio del ángulo
            if flag:
                ang = 360 - ang
        else:
            y = ((p2[1]-p1[1])*(p3[0]-p1[0])/(p2[0]-p1[0]))+p1[1]
            #Si está la tercera parte por encima de la recta que hacen las dos primera entonces se cambia el angulo
            if p3[1] > y:
                ang = 360 - ang
        
        return ang

### Funciones de comparación y ejecución

In [4]:
#Versión 4 de la comparación
def compararPosiciones(pos1,pos2,pesos={"brazos":1,"piernas":1,"torso":1}):
    zonas={"brazos":["angCodo","angHombro"],"piernas":["angRodilla","angCadera"],"torso":["angCaderaTorso","angCuelloSup"]}
    res=0
    total=0
    
    #Se recoge el peso total
    for i in pesos:
        if i not in zonas:
            raise Exception("No se puede dar peso a una zona que no esté definida")
        total+=pesos[i]
        
    #Se recorren los distintos tipos de zonas y se les aplica el peso a la comparación
    for i in zonas:
        res+=(pesos[i]/total)*comparacionZona(pos1,pos2,zonas[i])
    
    porcentaje = res*100/180
    
    return res,100-porcentaje

In [5]:
def comparacionZona(pos1,pos2,zonas):
    partes=["D","I"]
    res = 0.0
    for i in partes:
        for j in zonas:
            #aux es la diferencia entre los ángulos
            aux = abs(eval("pos1."+j+i)-eval("pos2."+j+i))
            #si la diferencia es mayor de 180 grados se coge el otro lado
            if aux > 180:
                res += (360-aux)
            else:
                res+=aux
    return res/(len(partes)*len(zonas))

In [6]:
def obtenerFrame(vid,nframe,path='../pruebas/videos-prueba/Videos/Josemi'):
    vc = cv2.VideoCapture(path+vid)

    if (vc.isOpened()==False):
        print("Error")
    else:
        for i in range(frame):
            ret,nframe = vc.read()

    o = predictor(frame)
    v = Visualizer(frame[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v2 = v.draw_instance_predictions(o["instances"].to("cpu"))
    imVisualizer = cv2.cvtColor(v2.get_image()[:, :, ::-1],cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(18, 16))
    plt.imshow(imVisualizer)
    plt.show()
    
    pkP = o.get("instances").pred_keypoints
    if len(pkP)>0:
        x = pkP[0][:,0].cpu().numpy()
        y = pkP[0][:,1].cpu().numpy()
    return [x,y]

In [7]:
def obtenerPosiciones(path = '../pruebas/videos-prueba/Videos/Josemi'):
    files=[]
    pos={}
    tiempos={}
    for r, d, f in os.walk(path):
        for file in f:
            print("Obteniendo posición del vídeo " + file)
            startPos = dt.now().timestamp()*1000
            pos[file]=[]
            vc = cv2.VideoCapture(path + "/" + file)

            if (vc.isOpened()==False):
                print("Error")

            (grabbed, frame) = vc.read()
            i=0
            while(vc.isOpened()):
                ret,frame = vc.read()
                if ret == True:
                    o = predictor(frame)
                    pkP = o.get("instances").pred_keypoints
                    if len(pkP)>0 and len(pkP[0])==17:
                        x = pkP[0][:,0].cpu().numpy()
                        y = pkP[0][:,1].cpu().numpy()
                        pos[file].append(Posicion(x,y))
                else:
                    break
                print("Frame " + str(i),end="\r")
                i+=1
                
            vc.release()
            cv2.destroyAllWindows()
            stopPos = dt.now().timestamp()*1000
            tiempos[file]=stopPos-startPos
            print("Fin de la obtención de posición del vídeo " + file + "\n")
    
    print("\n\n")
    return pos, tiempos

In [8]:
def compararFrames(posiciones,pesos={"brazos":1,"piernas":1,"torso":1}):
    sol = {}
    por = {}
    tiempos={}
    for j in posiciones:
        print("Comparando frames del vídeo " + j)
        startCom = dt.now().timestamp()*1000
        sol[j]=[]
        por[j]=[]
        for i in range(len(posiciones[j])):
            if i+1 < len(posiciones[j]):
                res,porcentaje=compararPosiciones(posiciones[j][i],posiciones[j][i+1],pesos)
                sol[j].append(res)
                por[j].append(porcentaje)
        stopCom = dt.now().timestamp()*1000
        tiempos[j]=stopCom-startCom
        print("Fin de comparación del vídeo " + j + "\n")
    
    print("\n\n")
    return sol,por, tiempos

In [9]:
def estadisticas(puntuaciones,porcentajes):
    media={}
    mediaPorc={}
    nframes={}
    maximos={}
    maximosPorc={}
    minimos={}
    minimosPorc={}
    desv={}
    desvPorc={}
    tiempos={}
    for i in puntuaciones:
        print("Obteniendo estadísticas de " + i)
        startPunt = dt.now().timestamp()*1000
        aux = puntuaciones[i]
        auxPorc = porcentajes[i]
        media[i]= sum(aux)/len(aux)
        mediaPorc[i]= sum(auxPorc)/len(auxPorc)
        maximos[i]=max(aux)
        maximosPorc[i]=max(auxPorc)
        minimos[i]=min(aux)
        minimosPorc[i]=min(auxPorc)
        desv[i]=np.std(aux)
        desvPorc[i]=np.std(auxPorc)
        nframes[i] = len(aux)
        stopPunt = dt.now().timestamp()*1000
        tiempos[i]=stopPunt-startPunt
        print("Fin de la obtención de estadísticas de " + i + "\n")
    return media,maximos,minimos,desv,mediaPorc,maximosPorc,minimosPorc,desvPorc,nframes, tiempos

In [10]:
def procesarVideos(path = '../pruebas/videos-prueba/Videos/Josemi',pesos={"brazos":1,"piernas":1,"torso":1}):
    
    pos,difPos = obtenerPosiciones(path)
    
    sol,por, difCom = compararFrames(pos,pesos)
    
    media,maximos,minimos,desv,mediaPorc,maximosPorc,minimosPorc,desvPorc,nframes,difPunt = estadisticas(sol,por)
    
    
    tiempos=[]
    for i in media:
        tiempos.append([i,media[i],maximos[i],minimos[i],desv[i],mediaPorc[i],maximosPorc[i],minimosPorc[i],desvPorc[i],nframes[i],difPos[i],difCom[i],difPunt[i]])
        
    return pd.DataFrame(tiempos)

In [11]:
pandas= procesarVideos()

Obteniendo posición del vídeo depie.webm
Fin de la obtención de posición del vídeo depie.webm

Obteniendo posición del vídeo sentado1.webm
Fin de la obtención de posición del vídeo sentado1.webm

Obteniendo posición del vídeo sentado6-camiseta.webm
Fin de la obtención de posición del vídeo sentado6-camiseta.webm

Obteniendo posición del vídeo sentado2-cruzado-480.webm
Fin de la obtención de posición del vídeo sentado2-cruzado-480.webm

Obteniendo posición del vídeo sentado4-remangado.webm
Fin de la obtención de posición del vídeo sentado4-remangado.webm

Obteniendo posición del vídeo sentado2-cruzado.webm
Fin de la obtención de posición del vídeo sentado2-cruzado.webm

Obteniendo posición del vídeo sentado3-caja.webm
Fin de la obtención de posición del vídeo sentado3-caja.webm

Obteniendo posición del vídeo sentado5-chaqueta-abierta.webm
Fin de la obtención de posición del vídeo sentado5-chaqueta-abierta.webm




Comparando frames del vídeo depie.webm
Fin de comparación del vídeo depie

In [12]:
pandas.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,depie.webm,5.258437,34.893516,0.460191,5.926812,97.078646,99.744339,80.614713,3.292673,258,26735.743896,50.474121,0.206055
1,sentado1.webm,2.677703,26.068990,0.054611,3.014065,98.512387,99.969661,85.517228,1.674481,445,50953.609131,80.244873,0.163818
2,sentado6-camiseta.webm,2.364012,5.270336,0.324795,0.960030,98.686660,99.819558,97.072036,0.533350,194,21334.305176,31.359131,0.113037
3,sentado2-cruzado-480.webm,2.737517,14.589717,0.578862,1.718944,98.479157,99.678410,91.894601,0.954969,286,31126.146973,44.600830,0.124023
4,sentado4-remangado.webm,2.780224,23.798344,0.222482,2.964205,98.455431,99.876399,86.778698,1.646780,219,24638.309082,34.361816,0.112061


In [13]:
pandas = pandas.rename(columns={0:"Vídeo",1:"MediaGrados",2:"GradosMáximos",3:"GradosMínimos",4:"DesviaciónGrados",5:"PorcentajeMedio",6:"PorcentajeMáximo",7:"PorcentajeMínimo",8:"DesviaciónPorcentaje",9:"NúmeroFrames",10:"TiempoObtPosición",11:"TiempoComparación",12:"TiempoEstadísticas"})
pandas.head()

,Vídeo,MediaGrados,GradosMáximos,GradosMínimos,DesviaciónGrados,PorcentajeMedio,PorcentajeMáximo,PorcentajeMínimo,DesviaciónPorcentaje,NúmeroFrames,TiempoObtPosición,TiempoComparación,TiempoEstadísticas
0,depie.webm,5.258437,34.893516,0.460191,5.926812,97.078646,99.744339,80.614713,3.292673,258,26735.743896,50.474121,0.206055
1,sentado1.webm,2.677703,26.068990,0.054611,3.014065,98.512387,99.969661,85.517228,1.674481,445,50953.609131,80.244873,0.163818
2,sentado6-camiseta.webm,2.364012,5.270336,0.324795,0.960030,98.686660,99.819558,97.072036,0.533350,194,21334.305176,31.359131,0.113037
3,sentado2-cruzado-480.webm,2.737517,14.589717,0.578862,1.718944,98.479157,99.678410,91.894601,0.954969,286,31126.146973,44.600830,0.124023
4,sentado4-remangado.webm,2.780224,23.798344,0.222482,2.964205,98.455431,99.876399,86.778698,1.646780,219,24638.309082,34.361816,0.112061


In [14]:
pandas["TiempoTotal"]=pandas["TiempoObtPosición"]+pandas["TiempoComparación"]+pandas["TiempoEstadísticas"]
pandas["Tiempo/Frame"]=pandas["TiempoTotal"]/pandas["NúmeroFrames"]
pandas.head()

,Vídeo,MediaGrados,GradosMáximos,GradosMínimos,DesviaciónGrados,PorcentajeMedio,PorcentajeMáximo,PorcentajeMínimo,DesviaciónPorcentaje,NúmeroFrames,TiempoObtPosición,TiempoComparación,TiempoEstadísticas,TiempoTotal,Tiempo/Frame
0,depie.webm,5.258437,34.893516,0.460191,5.926812,97.078646,99.744339,80.614713,3.292673,258,26735.743896,50.474121,0.206055,26786.424072,103.823349
1,sentado1.webm,2.677703,26.068990,0.054611,3.014065,98.512387,99.969661,85.517228,1.674481,445,50953.609131,80.244873,0.163818,51034.017822,114.683186
2,sentado6-camiseta.webm,2.364012,5.270336,0.324795,0.960030,98.686660,99.819558,97.072036,0.533350,194,21334.305176,31.359131,0.113037,21365.777344,110.132873
3,sentado2-cruzado-480.webm,2.737517,14.589717,0.578862,1.718944,98.479157,99.678410,91.894601,0.954969,286,31126.146973,44.600830,0.124023,31170.871826,108.989062
4,sentado4-remangado.webm,2.780224,23.798344,0.222482,2.964205,98.455431,99.876399,86.778698,1.646780,219,24638.309082,34.361816,0.112061,24672.782959,112.661109


In [15]:
pandas

,Vídeo,MediaGrados,GradosMáximos,GradosMínimos,DesviaciónGrados,PorcentajeMedio,PorcentajeMáximo,PorcentajeMínimo,DesviaciónPorcentaje,NúmeroFrames,TiempoObtPosición,TiempoComparación,TiempoEstadísticas,TiempoTotal,Tiempo/Frame
0,depie.webm,5.258437,34.893516,0.460191,5.926812,97.078646,99.744339,80.614713,3.292673,258,26735.743896,50.474121,0.206055,26786.424072,103.823349
1,sentado1.webm,2.677703,26.068990,0.054611,3.014065,98.512387,99.969661,85.517228,1.674481,445,50953.609131,80.244873,0.163818,51034.017822,114.683186
2,sentado6-camiseta.webm,2.364012,5.270336,0.324795,0.960030,98.686660,99.819558,97.072036,0.533350,194,21334.305176,31.359131,0.113037,21365.777344,110.132873
3,sentado2-cruzado-480.webm,2.737517,14.589717,0.578862,1.718944,98.479157,99.678410,91.894601,0.954969,286,31126.146973,44.600830,0.124023,31170.871826,108.989062
4,sentado4-remangado.webm,2.780224,23.798344,0.222482,2.964205,98.455431,99.876399,86.778698,1.646780,219,24638.309082,34.361816,0.112061,24672.782959,112.661109
5,sentado2-cruzado.webm,2.789076,15.350412,0.397699,1.812059,98.450513,99.779056,91.471993,1.006699,286,32914.017822,44.590088,0.128906,32958.736816,115.240339
6,sentado3-caja.webm,2.619265,34.322536,0.145171,4.125018,98.544853,99.919349,80.931924,2.291676,299,33129.628174,46.426025,0.125977,33176.180176,110.957124
7,sentado5-chaqueta-abierta.webm,2.961432,17.293844,0.423034,1.941387,98.354760,99.764981,90.392309,1.078549,281,31442.279297,44.332031,0.123047,31486.734375,112.052435


In [16]:
print(pandas.to_latex())

\begin{tabular}{llrrrrrrrrrrrrrr}
\toprule
{} &                           Vídeo &  MediaGrados &  GradosMáximos &  GradosMínimos &  DesviaciónGrados &  PorcentajeMedio &  PorcentajeMáximo &  PorcentajeMínimo &  DesviaciónPorcentaje &  NúmeroFrames &  TiempoObtPosición &  TiempoComparación &  TiempoEstadísticas &   TiempoTotal &  Tiempo/Frame \\
\midrule
0 &                      depie.webm &     5.258437 &      34.893516 &       0.460191 &          5.926812 &        97.078646 &         99.744339 &         80.614713 &              3.292673 &           258 &       26735.743896 &          50.474121 &            0.206055 &  26786.424072 &    103.823349 \\
1 &                   sentado1.webm &     2.677703 &      26.068990 &       0.054611 &          3.014065 &        98.512387 &         99.969661 &         85.517228 &              1.674481 &           445 &       50953.609131 &          80.244873 &            0.163818 &  51034.017822 &    114.683186 \\
2 &          sentado6-camiseta.webm & 

## Cálculo del tiempo medio de procesamiento por fotograma

In [17]:
pandas["Tiempo/Frame"].mean()

111.06743477781936